In [ ]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key_1 = 'Demos/churn-bigml-80.csv'
file_key_2 = 'Demos/churn-bigml-20.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

## Reading the csv files
telecom_train = pd.read_csv(file_content_stream_1)
telecom_test = pd.read_csv(file_content_stream_2)

In [ ]:
## Changing logical value to numbers
telecom_train['Churn_numb'] = np.where(telecom_train['Churn'] == False, 0, 1)
telecom_test['Churn_numb'] = np.where(telecom_test['Churn'] == False, 0, 1)

telecom_train['International_plan'] = np.where(telecom_train['International_plan'] == 'No', 0, 1)
telecom_test['International_plan'] = np.where(telecom_test['International_plan'] == 'No', 0, 1)

telecom_train['Voice_mail_plan'] = np.where(telecom_train['Voice_mail_plan'] == 'No', 0, 1)
telecom_test['Voice_mail_plan'] = np.where(telecom_test['Voice_mail_plan'] == 'No', 0, 1)

telecom_train['total_charge'] = telecom_train['Total_day_charge'] + telecom_train['Total_eve_charge'] + telecom_train['Total_night_charge'] + telecom_train['Total_intl_charge']
telecom_test['total_charge'] = telecom_test['Total_day_charge'] + telecom_test['Total_eve_charge'] + telecom_test['Total_night_charge'] + telecom_test['Total_intl_charge']

telecom_train.head()

In [ ]:
## Selecting variables of interest
telecom_train = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls', 'Churn_numb']]
telecom_test = telecom_test[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls', 'Churn_numb']]

telecom_train.head()

In [ ]:
## Defining the input and target variables
X = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']]
Y = telecom_train['Churn_numb']

## Defining data-frame to store results 
RF_importances = pd.DataFrame({'Account_length': np.repeat(np.nan, 100), 'International_plan': np.repeat(np.nan, 100), 'Voice_mail_plan': np.repeat(np.nan, 100), 'total_charge': np.repeat(np.nan, 100), 'Customer_service_calls': np.repeat(np.nan, 100)})
Ada_importances = pd.DataFrame({'Account_length': np.repeat(np.nan, 100), 'International_plan': np.repeat(np.nan, 100), 'Voice_mail_plan': np.repeat(np.nan, 100), 'total_charge': np.repeat(np.nan, 100), 'Customer_service_calls': np.repeat(np.nan, 100)})
GB_importances = pd.DataFrame({'Account_length': np.repeat(np.nan, 100), 'International_plan': np.repeat(np.nan, 100), 'Voice_mail_plan': np.repeat(np.nan, 100), 'total_charge': np.repeat(np.nan, 100), 'Customer_service_calls': np.repeat(np.nan, 100)})

for i in range(0, 100):

    ## Splitting the data 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    ###################
    ## Random Forest ##
    ###################
    
    RF = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    RF_importances.loc[i] = RF.feature_importances_
    
    ##############
    ## AdaBoost ##
    ##############
    
    Ada = AdaBoostClassifier(n_estimators = 500, base_estimator = DecisionTreeClassifier(max_depth = 3), learning_rate = 0.01).fit(X_train, Y_train)
    Ada_importances.loc[i] = Ada.feature_importances_
    
    #######################
    ## Gradient Boosting ##
    #######################
    
    GB = GradientBoostingClassifier(n_estimators = 500, max_depth = 3, learning_rate = 0.01).fit(X_train, Y_train)
    GB_importances.loc[i] = GB.feature_importances_
    
## Combining the importances of the three models 
Feature_Importances = pd.concat([RF_importances, Ada_importances, GB_importances], axis = 0)
Feature_Importances.mean()

In [ ]:
Feature_Importances = pd.concat([RF_importances, Ada_importances, GB_importances], axis = 0)
Feature_Importances.mean()

In [ ]:
## Re-defining the input and target variables
X = telecom_train[['Account_length', 'International_plan', 'total_charge', 'Customer_service_calls']]
Y = telecom_train['Churn_numb']
